# FiftyOne

v221210 -> v221212(gun)

### 이미지 검수
1. UFO 형식 어노테이션/이미지 불러오기
2. 이미지 / 어노테이션 검수 (FiftyOne)
3. 라벨 수정 (CVAT 연동)
4. 라벨 내보내기


### pred_gt 비교
1. UFO 형식 어노테이션/이미지 불러오기
2. 이미지 / 어노테이션 검수 (FiftyOne)

### utils
1. fiftyone dataset 목록 / 가져오기 / 삭제




In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.core.labels as fol
from fiftyone import ViewField as F

import json
import os
import os.path as osp
from pathlib import Path
from random import randint
from itertools import combinations, compress
import requests

import pandas as pd
import numpy as np
from rich import print
from tqdm.notebook import tqdm

from utils.ufo import UFODataset, label_schema



# 이미지 검수
#### 1. UFO 형식 어노테이션/이미지 불러오기

In [ ]:
# 정해줘야 하는 값들
# 이미지 파일 경로
DATA_PATH = '/opt/ml/input/data/ICDAR17_Korean/images/'

# 어노테이션 경로
LABELS_PATH = '/opt/ml/input/data/ICDAR17_Korean/ufo/train.json'



dataset = fo.Dataset.from_dir(
    name = 'my-dataset',
    data_path = DATA_PATH,
    labels_path = LABELS_PATH,
    dataset_type = UFODataset,
    include_all_data = True,
)

# 자신이 검수할 데이터셋 범위 지정
subset = dataset[100:200]

#### 2. 이미지 / 어노테이션 검수 (FiftyOne)

세션 실행

https://<서버 IP>:30001 형식의 주소로 접속해서 검수 가능

수정할 라벨들을 선택하면 됨. `<ESC>` 누르면 선택 취소되므로 주의

In [ ]:
PORT = 30001

session = fo.Session(subset, port=PORT, address='0.0.0.0', auto=False)

print(f'http://{requests.get("http://icanhazip.com").text.strip()}:{PORT}')

#### 3. 라벨 수정 (CVAT 연동)

FiftyOne에서 선택한 라벨들을 CVAT에 업로드. CVAT 웹사이트에서 라벨 수정 가능

In [ ]:
selected_view = dataset.select(session.selected)

print('선택한 이미지 개수:', len(selected_view))
assert len(selected_view) > 0, '검수할 이미지가 없음'

anno_key = 'a' + str(randint(0, 1e9))
print('anno_key:', anno_key)

selected_view.annotate(
    anno_key,
    label_schema = label_schema,
    launch_editor = True,
)

#### 4. 라벨 내보내기

CVAT에서 수정한 라벨 저장했는지 확인하고, 불러온 뒤, UFO 형식으로 저장

In [ ]:
dataset.load_annotations(anno_key)

In [ ]:
# 수정한 라벨만 저장하려면 selected_view
# 전체 데이터셋을 저장하려면 dataset
# 본인이 검수한 범위만 저장하려면 subset

selected_view.export(
    labels_path = 'outs/fixed.json',
    dataset_type=UFODataset,
)

# pred_gt 비교
#### 1. UFO 형식 어노테이션/이미지 불러오기


In [ ]:
# 정해줘야 하는 값들
# 이미지 파일 경로
DATA_PATH = '/opt/ml/input/data/ICDAR17_Korean/images/'

# 어노테이션 경로
LABELS_PATH = '/opt/ml/input/data/ICDAR17_Korean/ufo/train.json'

# 예측 경로
PRED_PATH = '/opt/ml/input/data/ICDAR17_Korean/ufo/train.json'


#gt
dataset = fo.Dataset.from_dir(
    name = 'my-dataset_gt',
    data_path = DATA_PATH,
    labels_path = LABELS_PATH,
    label_field="ground_truth",
    dataset_type = UFODataset,
)

#pred
dataset_pred = fo.Dataset.from_dir(
    name = 'my-dataset_pred',
    data_path = DATA_PATH,
    labels_path = PRED_PATH,
    label_field="predictions",
    dataset_type = UFODataset,
)

predictions_view = dataset_pred.select_fields("predictions")
dataset.merge_samples(predictions_view)


# 자신이 검수할 데이터셋 범위 지정
subset = dataset[100:200]

#### 2. 이미지 / 어노테이션 검수 (FiftyOne)

세션 실행

https://<서버 IP>:30001 형식의 주소로 접속해서 검수 가능

수정할 라벨들을 선택하면 됨. `<ESC>` 누르면 선택 취소되므로 주의

In [ ]:
PORT = 30001

session = fo.Session(subset, port=PORT, address='0.0.0.0', auto=False)

print(f'http://{requests.get("http://icanhazip.com").text.strip()}:{PORT}')

# Utils
#### 1. fiftyone dataset 목록 / 가져오기 / 삭제 


In [ ]:
# fiftyone dataset 목록
fo.list_datasets()

In [ ]:
# fiftyone dataset 가져오기
DATASET_NAME = "my-dataset"
dataset = fo.load_dataset(DATASET_NAME)

In [ ]:
# fiftyone dataset 삭제
dataset.delete()

In [ ]:
# fiftyone dataset 전체 삭제
for dataset_name in fo.list_datasets():
    fo.load_dataset(dataset_name).delete()